<a href="https://colab.research.google.com/github/diskfoexe/colab/blob/main/ComfyUI_Optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡ ComfyUI Optimized Colab

### 🔴 **Brought to you by [AI With Chucky](https://youtube.com/@AIWithChucky)**
*Subscribe for more AI tutorials, workflows, and optimization tips!*

**Features:**
- **Cloudflare Only**: Secure and fast external access.
- **Robust Storage**: Handles Google Drive disconnections automatically.
- **Smart Downloader**: Supports Checkpoints, Flux/Diffusion Models, Text Encoders, CLIP Vision, LoRAs, etc.
- **Auto-Manager**: ComfyUI Manager is always installed and up to date.

In [3]:
#@title 1. Environment Setup & ComfyUI Install
import os
import shutil
import subprocess

# --- Configuration ---
USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False   #@param {type:"boolean"}
# ---------------------

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    print("[INFO] Mounting Google Drive (Force Remount)...")
    try:
        drive.mount('/content/drive', force_remount=True)
        WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    except Exception as e:
        print(f"❌ [ERROR] Failed to mount Drive: {e}")
        print("Please restart the runtime: Runtime > Disconnect and Delete Runtime")
        raise
else:
    WORKSPACE = "/content/ComfyUI"

print(f"[INFO] Workspace set to: {WORKSPACE}")

# 1. Install/Update ComfyUI
if not os.path.exists(WORKSPACE):
    print("[INFO] Installing ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
else:
    if UPDATE_COMFY_UI:
        print("[INFO] Updating ComfyUI...")
        # Fix for previous crash: Remove git lock file if it exists
        lock_file = os.path.join(WORKSPACE, ".git", "index.lock")
        if os.path.exists(lock_file):
            print("[WARN] Found git lock file from previous crash. Removing...")
            os.remove(lock_file)

        try:
            # Check if directory is actually readable
            os.listdir(WORKSPACE)
            !cd {WORKSPACE} && git pull
        except OSError:
            print("❌ [ERROR] Google Drive is disconnected. Please restart Runtime.")

# 2. Install Dependencies
print("[INFO] Installing Dependencies...")
# Check if requirements file exists to confirm we are in the right place
if os.path.exists(os.path.join(WORKSPACE, "requirements.txt")):
    !cd {WORKSPACE} && pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
else:
    print(f"❌ [ERROR] requirements.txt not found in {WORKSPACE}. The drive might still be disconnected.")

print("\n[DONE] Environment Setup Complete.")

[INFO] Mounting Google Drive (Force Remount)...
Mounted at /content/drive
[INFO] Workspace set to: /content/drive/MyDrive/ComfyUI
[INFO] Installing Dependencies...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 22.5 MB/

In [4]:
#@title 2. Install ComfyUI Manager
import os

# Ensure WORKSPACE is defined
if 'WORKSPACE' not in locals():
    WORKSPACE = "/content/drive/MyDrive/ComfyUI" if os.path.exists("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"

MANAGER_PATH = os.path.join(WORKSPACE, "custom_nodes", "ComfyUI-Manager")

# Ensure custom_nodes directory exists
os.makedirs(os.path.dirname(MANAGER_PATH), exist_ok=True)

print(f"[INFO] Checking ComfyUI Manager at: {MANAGER_PATH}")

if os.path.exists(MANAGER_PATH):
    print("[INFO] Manager found. Checking for updates...")
    !cd {MANAGER_PATH} && git pull
else:
    print("[INFO] Manager not found. Cloning repository...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git {MANAGER_PATH}

# Verification Step
init_file = os.path.join(MANAGER_PATH, "__init__.py")
if os.path.exists(init_file):
    print("\n✅ [SUCCESS] ComfyUI Manager installed and verified successfully.")
else:
    # Loud Fail
    raise Exception("❌ [ERROR] ComfyUI Manager installation failed! Required files are missing.")

[INFO] Checking ComfyUI Manager at: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
[INFO] Manager found. Checking for updates...
Already up to date.

✅ [SUCCESS] ComfyUI Manager installed and verified successfully.


In [ ]:
#@title 3. Smart Model Downloader
#@markdown Paste direct download links below. This supports **Checkpoints, Diffusion Models (Flux/UNET), Text Encoders (CLIP/T5), CLIP Vision, VAEs, LoRAs, and ControlNets**.

import os
import requests
from tqdm.notebook import tqdm
from urllib.parse import urlparse, unquote
import re

# Ensure WORKSPACE is defined
if 'WORKSPACE' not in locals():
    WORKSPACE = "/content/drive/MyDrive/ComfyUI" if os.path.exists("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"

# --- Input Fields ---
CHECKPOINT_URLS = "" #@param {type:"string"}
UNET_DIFFUSION_URLS = "https://huggingface.co/Aryanne/acestep-v15-test-merges/resolve/main/acestep_v1.5_merge_sft_turbo_ta_0.4.safetensors?download=true" #@param {type:"string"}
TEXT_ENCODER_URLS = "https://huggingface.co/Comfy-Org/ace_step_1.5_ComfyUI_files/resolve/main/split_files/text_encoders/qwen_4b_ace15.safetensors?download=true" #@param {type:"string"}
CLIP_VISION_URLS = "" #@param {type:"string"}
VAE_URLS = "" #@param {type:"string"}
LORA_URLS = "" #@param {type:"string"}
CONTROLNET_URLS = "" #@param {type:"string"}
EMBEDDING_URLS = "" #@param {type:"string"}

# --- Directory Mapping ---
DIRS = {
    "checkpoints":      os.path.join(WORKSPACE, "models/checkpoints"),
    "diffusion_models": os.path.join(WORKSPACE, "models/diffusion_models"),
    "clip":             os.path.join(WORKSPACE, "models/clip"),
    "clip_vision":      os.path.join(WORKSPACE, "models/clip_vision"),
    "vae":              os.path.join(WORKSPACE, "models/vae"),
    "loras":            os.path.join(WORKSPACE, "models/loras"),
    "controlnet":       os.path.join(WORKSPACE, "models/controlnet"),
    "embeddings":       os.path.join(WORKSPACE, "models/embeddings")
}

for key, d in DIRS.items():
    os.makedirs(d, exist_ok=True)

def get_filename_from_headers(response, url):
    """Extract filename from Content-Disposition header or fallback to URL."""
    # 1. Try Content-Disposition
    if "Content-Disposition" in response.headers:
        cd = response.headers["Content-Disposition"]
        # Extract filename="example.safetensors"
        fnames = re.findall('filename="?([^"]+)"?', cd)
        if fnames:
            return fnames[0]

    # 2. Fallback to URL path
    parsed_url = urlparse(url)
    filename = unquote(os.path.basename(parsed_url.path))

    # 3. Last resort
    if not filename:
        return "downloaded_file.safetensors"

    return filename

def download_file_modern(url, target_dir):
    try:
        # Stream GET to catch headers before downloading
        response = requests.get(url, stream=True, allow_redirects=True)
        response.raise_for_status()

        filename = get_filename_from_headers(response, url)
        file_path = os.path.join(target_dir, filename)

        # Check if file exists
        if os.path.exists(file_path):
            # Optional: Check size if you wanted strictly smart resume, but simple skip is safer for Colab
            print(f"   ⏩ Skipping (Exists): {filename}")
            return

        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024 * 1024 # 1MB chunks

        print(f"   ⬇️ Downloading: {filename}")

        # Modern Progress Bar
        with tqdm(total=total_size, unit='iB', unit_scale=True, desc=filename, leave=False) as bar:
            with open(file_path, 'wb') as f:
                for data in response.iter_content(block_size):
                    bar.update(len(data))
                    f.write(data)

        print(f"   ✅ Downloaded: {filename}")

    except Exception as e:
        print(f"   ❌ Failed: {url} | Error: {e}")

def process_category(urls_str, target_dir, category_name):
    if not urls_str.strip():
        return

    url_list = [u.strip() for u in urls_str.replace(',', '\n').split('\n') if u.strip()]
    print(f"\n📂 [{category_name}] Processing {len(url_list)} files...")

    for url in url_list:
        download_file_modern(url, target_dir)

# --- Execute Downloads ---
process_category(CHECKPOINT_URLS,      DIRS["checkpoints"],      "Checkpoints")
process_category(UNET_DIFFUSION_URLS,  DIRS["diffusion_models"], "Diffusion Models (UNET/Flux)")
process_category(TEXT_ENCODER_URLS,    DIRS["clip"],             "Text Encoders (CLIP/T5)")
process_category(CLIP_VISION_URLS,     DIRS["clip_vision"],      "CLIP Vision")
process_category(VAE_URLS,             DIRS["vae"],              "VAE")
process_category(LORA_URLS,            DIRS["loras"],            "LoRA")
process_category(CONTROLNET_URLS,      DIRS["controlnet"],       "ControlNet")
process_category(EMBEDDING_URLS,       DIRS["embeddings"],       "Embeddings")

print("\n✅ [DONE] All downloads finished.")

In [1]:
# @title 4. NO Disconnect
%%html
<b>🔊 Keep-Alive Audio Player</b><br>
<audio src="https://raw.githubusercontent.com/anars/blank-audio/master/10-minutes-of-silence.mp3" autoplay loop controls style="width: 300px;" />
<br><small>Running this silent audio loop prevents the browser from throttling the connection when you switch tabs.</small>

In [ ]:
#@title 5. Run ComfyUI (ngrok - Faster Alternative)
import subprocess
import threading
import time
import socket
import os

# --- הגדרות ---
NGROK_TOKEN = "39cJ9cVd5MI3TakFJ0scLF2iAg6_35DXdM1pcjjpSKRTZ4rov" # <--- שים את הטוקן שלך פה
PORT = 8188
WORKSPACE = "/content/drive/MyDrive/ComfyUI" if os.path.exists("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"

# 1. התקנת ngrok
if not os.path.exists('/usr/local/bin/ngrok'):
    print("[INFO] Installing ngrok...")
    !curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
    !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
    !sudo apt update && sudo apt install ngrok
    !ngrok config add-authtoken {NGROK_TOKEN}

# 2. לוגיקה להפעלת המנהרה
def start_ngrok(port):
    while True:
        time.sleep(1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\n[INFO] ComfyUI detected. Launching ngrok tunnel...")
    # סגירת תהליכי ngrok קודמים אם קיימים
    !pkill ngrok
    # הפעלה
    get_ipython().system_raw(f'ngrok http {port} &')

    time.sleep(2)
    # שליפת הכתובת הציבורית
    import json
    import requests
    try:
        response = requests.get('http://localhost:4040/api/tunnels')
        url = response.json()['tunnels'][0]['public_url']
        print("\n" + "="*60)
        print(f"🚀 PUBLIC URL (ngrok): {url}")
        print("="*60 + "\n")
    except:
        print("❌ Failed to get ngrok URL. Check your token.")

# 3. הפעלת ה-Thread
threading.Thread(target=start_ngrok, daemon=True, args=(PORT,)).start()

# 4. הרצת ComfyUI
if os.path.exists(os.path.join(WORKSPACE, "main.py")):
    %cd {WORKSPACE}
    print("[INFO] Booting ComfyUI...")
    !python main.py --dont-print-server --listen 127.0.0.1 --port {PORT}
else:
    print(f"❌ [ERROR] Could not find main.py in {WORKSPACE}")